## Remap file creation for mizuRoute

The following code is to prepare the remap for easy more from structured or unstrcutures mesh to river network routing subbasins.

The user should call easymore nc_remapper for target shapefile, in this case river network subbasins and a smaple of netcdf file file for runoff field.

If the target river network topology is large or is separated into various units such as continent, users can make the remap file for each part of river network topology first, then merge the remap file into one large file and then create the remap nc file from that.


In [ ]:
# loading EASYMORE
from easymore.easymore import easymore
# initializing EASYMORE object
esmr = easymore()

# specifying EASYMORE objects
# name of the case; the temporary, remapping and remapped file names include case name
esmr.case_name                = 'CLM_HDMA'
# temporary path that the EASYMORE generated GIS files and remapped file will be saved
esmr.temp_dir                 = '../temporary/'
# name of target shapefile that the source netcdf files should be remapped to
esmr.target_shp               = '/Volumes/F:/hydrography/test/hydrolake_sub100km_final_corrected_basins_1000000.shp'
esmr.target_shp_ID            = 'ID' # the name of the field that identify the river network subbasins
# name of netCDF file(s); multiple files can be specified with *
esmr.source_nc                = '../data/CLM/CLM_sample.nc' # a sample from CLM nc runoff file
# name of variables from source netCDF file(s) to be remapped
esmr.var_names                = ['QRUNOFF']
# name of variable longitude in source netCDF files
esmr.var_lon                  = 'lon'
# name of variable latitude in source netCDF files
esmr.var_lat                  = 'lat'
# name of variable time in source netCDF file; should be always time
esmr.var_time                 = 'time'
# location where the remapped netCDF file will be saved
esmr.output_dir               = '../output/'

# execute EASYMORE
esmr.nc_remapper()

# Creation of the remap file

user should provide the name of the remapping csv file. if run with the pervious block of calling easymore, no change in the code is needed.


In [ ]:
# create the remapp file 
# read the easymore remapp file for the given case
import pandas as pd
import netCDF4 as nc4
import numpy as np
import xarray as xr
import os

# read the remapp file generated by easymore
df = pd.read_csv(esmr.temp_dir+esmr.case_name+'_remapping.csv') # read the remap file based on the 
# df = pd.read_csv('temporary_folder_case_name_remapping.csv') # read the remap file based on the 

# sort and get the case
df = df.sort_values(by=['ID_t']) # sort based on ID_t
case = df['easymore_case'].iloc[0].item()

# river network ID and its frequency in intersection with runoff field grid
RN_id, RN_frequency_in_intersection = np.unique(df['ID_t'], return_counts=True)
print("the dimension of overlap between river network and hydrological units:")
print(len(df))
if len(RN_id) == len(RN_frequency_in_intersection):
    print("the dimension of river network and its frequancy are:")
    print(len(RN_id))
    print(len(RN_frequency_in_intersection))
else:
    sys.exit("the dimension of river network and its frequency are not the same")

# get remapping information from easymore remap file
IDmask  = np.array(df['ID_t']) # the ID of river network
weight  = np.array(df['weight']) # the weight of each hydrological unit in river network
i_index = np.array(df['cols']) # cols for case 1 and 2
j_index = np.array(df['rows']) # rows for case 1 and 2
ID_s    = np.array(df['ID_s']) # ID from unstructure mesh case 3


# write the netcdf file
os.remove('remapping.nc')

with nc4.Dataset('remapping.nc', "w", format="NETCDF4") as ncid: # creating the NetCDF file
    
    # define the dimensions
    dimid_ID  = ncid.createDimension('polyid', len(RN_id))  # limited dimensiton equal the number of hruID
    dimid_RMP = ncid.createDimension('intersect', len(df))   # intersection
    
    # Variables RN_id, RN_frequency_in_intersection
    #
    RN_IDvar   = ncid.createVariable('RN_ID', 'int',  ('polyid', ), fill_value = -9999)
    RN_IDvar.long_name = 'ID of River Network subbasins'
    RN_IDvar.standard_name = 'ID of River Network subbasins'
    RN_IDvar.units = '1'
    RN_IDvar[:] = RN_id
    #
    RN_FRvar   = ncid.createVariable('RN_FR', 'int', ('polyid', ), fill_value = -9999)
    RN_FRvar.long_name = 'Frequancy of intersection River Network subbasins with hydrological subbasins'
    RN_FRvar.standard_name = 'Frequancy of intersection River Network subbasins with hydrological subbasins'
    RN_FRvar.units = '1'
    RN_FRvar[:] = RN_frequency_in_intersection
    
    # ID_mask, weight, i_index, j_index, ID_s
    #
    IDmaskvar  = ncid.createVariable('IDmask', 'int',  ('intersect', ), fill_value = -9999)
    IDmaskvar.long_name = 'ID of rive network subbasins'
    IDmaskvar.standard_name = 'ID of rive network subbasins'
    IDmaskvar.units = '1'
    IDmaskvar[:] = IDmask
    #
    weightvar   = ncid.createVariable('weight', 'f8',  ('intersect', ), fill_value = -9999)
    weightvar.long_name = 'weight of each hydrological unit in river network subbasins'
    weightvar.standard_name = 'weight of each hydrological unit in river network subbasins'
    weightvar.units = '1'
    weightvar[:] = weight
    
    if case == 1 or case == 2:
        #
        i_indexvar  = ncid.createVariable('i_index', 'int',  ('intersect', ), fill_value = -9999)
        i_indexvar.long_name = 'cols from the source nc file'
        i_indexvar.standard_name = 'cols from the source nc file'
        i_indexvar.units = '1'
        i_indexvar[:] = i_index + 1
        #
        j_indexvar   = ncid.createVariable('j_index', 'int',  ('intersect', ), fill_value = -9999)
        j_indexvar.long_name = 'rows from the source nc file'
        j_indexvar.standard_name = 'rows from the source nc file'
        j_indexvar.units = '1'
        j_indexvar[:] = j_index + 1
        
    if case == 3:
        #
        ID_HRvar   = ncid.createVariable('ID_HR', 'int',  ('intersect', ), fill_value = -9999)
        ID_HRvar.long_name = 'river network ID'
        ID_HRvar.standard_name = 'river network ID'
        ID_HRvar.units = '1'
        ID_HRvar[:] = ID_s
    
    # general attributes for NetCDF file
    ncid.Conventions = 'CF-1.6'
    ncid.Author = 'The data were written by easymore_codes'
    ncid.License = 'MIT'
    ncid.History = 'Created '
    ncid.Source = 'Case: ; remapped by script from library of Shervan Gharari (https://github.com/ShervanGharari/EASYMORE).'
    

# show the remap file
ds = xr.open_dataset('remapping.nc')
print(ds)
